In [2]:
pip install lightfm

In [3]:
#Загрузим необходимые для работы библиотеки
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

#Для визуализации
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Для моделирования
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score


In [4]:
#Считаем файлы
train = pd.read_csv('data/train_pr1.csv')
test = pd.read_csv('data/test.csv')
sample = pd.read_csv('data/sample_submission.csv')

In [7]:
items = pd.read_csv('data/items1.csv')

# 3 Создание модели

## 3.1 Строим матрицу взаимодействий, строки: userid; столбцы: itemid

In [12]:
# построим разреженную матрицу взаимодействий
ratings_coo = sparse.coo_matrix((train['rating'].astype(int),
                                 (train['userid'],train['itemid'])))

train_rating_coo, test_rating_coo = random_train_test_split(ratings_coo)

## 3.2 Строим разреженную матрицу признаков по товарам

Для этого используем следующие данные:

1 Категорию продукта 

2 Общее количество отзывов на каждый продукт

3 Средняя оценка по 5-ти балльной шкале

In [13]:
# Приведем count_rev к шкале от 0 до 5, чтобы сделать соразмерным оценке Overall
items['count_rev'] = 5 * items['count_rev']/items['count_rev'].max()

In [14]:
# Обработка признаков товаров
def item_feats(df_items):
    N_items=df_items.item_id.max() + 1
    N_cat = df_items.cat_id.max() + 1
    
    # данные, которые будут храниться в разреженной матрице:
    # 1-категори (передаем единицы, которые будут соответствовать столбцу с номером категории)
    # 2 - count_rev - количеcтво отзывов на товар
    # 3 - overall_mean -средняя оценка по 5-ти балльной шкале
    
    data = np.concatenate([np.array([1] * len(df_items)), 
                           df_items.count_rev.values,
                           df_items.overall_mean.values])
    
    # номера строк - повторим трижды список item_id
    rows_ids = np.tile(df_items.item_id, 3)
    
    # номера колонок
    # 1-номер категории
    # 2 - одна колонка, номер = макс. категорией + 1
    # 3 - одна колонка, номер = макс. категорией + 2
    col1 = df_items.cat_id.values
    col2 = np.array([N_cat + 1] * len(df_items))
    col3 = np.array([N_cat + 2] * len(df_items))
    cols_ids = np.concatenate((col1, col2, col3))
    
    return sparse.coo_matrix((data, (rows_ids,cols_ids)))
    
item_features = item_feats(items)
item_features

<41320x1178 sparse matrix of type '<class 'numpy.float64'>'
	with 123903 stored elements in COOrdinate format>

## 3.3 Построим пробную модель из baseline

In [15]:
NUM_THREADS = 2 #число потоков
NUM_COMPONENTS = 20 #число параметров вектора 
NUM_EPOCHS = 10 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(train_rating_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [16]:
#  Сделаем предсказание на тестовой выборке
preds = model.predict(test_rating_coo.row,
                      test_rating_coo.col)

In [17]:
# Посмотрим на метрику roc_auc_score
sklearn.metrics.roc_auc_score(test_rating_coo.data,preds)

0.7253273346812417

In [18]:
# Посчитаем precision, recall.
pr_at_k = precision_at_k(model=model, test_interactions=test_rating_coo, k=5)
rc_at_k = recall_at_k(model=model, test_interactions=test_rating_coo, k=5)
print(f'Precision: {pr_at_k.mean()}, recall: {rc_at_k.mean()}')

Precision: 0.01301596686244011, recall: 0.04021835242595316


In [19]:
# Сделаем предсказание на Kaggle
# Для этого обучим модель на всех имеющихся данных
model_kaggle = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model_kaggle = model_kaggle.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)
preds_kaggle = model_kaggle.predict(test.userid.values,
                                 test.itemid.values)
preds_kaggle = (preds_kaggle - preds_kaggle.min())/(preds_kaggle.max() - preds_kaggle.min())
sample['rating'] = preds_kaggle
sample.to_csv('submission_base.csv', index=False)

Промежуточный итог¶
Базовая модель работает, найдены метрики для нее, можно делать предсказания.
RocAuc для тестовых данных: 0.725 (sklern)
для Каггл: 0.76229
Вычисленные метрики : RocAuc, Precision, Recall. Можно также получить эмбединги и биасы. следующий этап -  подбор оптимальных параметров модели

## 3.4 Подбор параметров модели

Добавим теперь к этой модели признаки товаров и регуляризацию. По результатам  экспериментов получили, что лучшими параметрами будут:

In [38]:
NUM_THREADS = 4 #число потоков
NUM_COMPONENTS = 40
NUM_EPOCHS = 10
LR = 0.1
alphas = 0.00001
LOSS_TYPE = 'logistic'

In [39]:
#Создаём модель с регуляриацией и использованием признаков товаров
model_reg = LightFM(learning_rate=LR, loss=LOSS_TYPE, no_components=NUM_COMPONENTS)
                   
#Обучаем модель
model_reg = model_reg.fit(train_rating_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS,
                  item_features=item_features)

# сделаем предсказание для тестовой выборки
preds = model_reg.predict(test_rating_coo.row,
                     test_rating_coo.col,
                     item_features=item_features)

# RocAuc из sklern:
sklearn.metrics.roc_auc_score(test_rating_coo.data, preds)

0.7134180114772288

Делаем предсказание на тестовой выборке для Каггл

In [40]:
#Создаём модель с регуляриацией для Kaggle 
model_reg_kaggle = LightFM(learning_rate=LR, loss=LOSS_TYPE, no_components=NUM_COMPONENTS,
                   item_alpha=alphas, user_alpha=alphas)

#Обучаем модель
model_reg_kaggle = model_reg_kaggle.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS,
                  item_features=item_features)

preds_kaggle = model_reg_kaggle.predict(test.userid.values,
                     test.itemid.values,
                  item_features=item_features)

preds_kaggle = (preds_kaggle - preds_kaggle.min())/(preds_kaggle.max() - preds_kaggle.min())
sample['rating'] = preds_kaggle
sample.to_csv('submission_base1.csv', index=False)

# 4 Рекомендации для пользователя
Для существующх пользователей дадим рекомендации на основании его покупок, а новым пользователям предложим наиболее популярные товары

In [41]:
# для прототипа создадим сокращенную версию тренировочного набора:
train_reduced = train[['userid', 'itemid', 'overall', 'reviewTime']]

N_last = 5 #столько последних покупок будем показывать и к ним давать рекомендации
# Возьмем произвольно номер пользователя, для которого будем давать рекомендации
user_id_for_recommend = 2052

In [42]:
# получим эмбеддинги и запишем их в файл
item_biases, item_embeddings = model_reg_kaggle.get_item_representations(features=item_features)
user_biases, user_embeddings = model_reg_kaggle.get_user_representations()
user_embeddings.max(), item_embeddings.max()

(0.4239724, 14.608872)

In [27]:
pip install nmslib

     |████████████████████████████████| 13.0MB 317kB/s 
     |████████████████████████████████| 296kB 47.4MB/s 


In [43]:
import nmslib
 
#Создаём граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Добавляем товары в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [44]:
#Вспомогательная функция для поиска по графу
def nearest_item_nms(item_id, index, n=5):
    nn = index.knnQuery(item_embeddings[item_id], k=n)
    return nn

In [45]:
items_recommend_ids = set()
if user_id_for_recommend in train_reduced['userid'].unique():
    
    #выводим весь список покупок для этого пользователя
    train_reduced[train_reduced['userid'] == user_id_for_recommend]
    #список трех последних покупок, оцененных на макс оценку
    last_best_items = train_reduced[train_reduced['userid'] == user_id_for_recommend
                               ].sort_values(by = ['overall', 'reviewTime'], 
                               ascending=False)['itemid'].values[:N_last]
    print(f'Last best items are:')
    for item_id in last_best_items:
        print (items[items['item_id'] == item_id].title)

    # составляем список рекомендаций к последним покупкам
    for item in last_best_items:
        items_recommend_ids.update(nearest_item_nms(item,nms_idx)[0])
else:
    # новым пользователям предложим случайные 10 товаров из числа тех, которые получили оценки 4 и 5 
    # и количество отзывов на них в верхних 20%
    
    print("It's a new user!")
    most_popular_items = items[(items['NoRev'] >= items['NoRev'].quantile(0.95))]
    items_recommend_ids.update(most_popular_items.sample(10)['itemid'].values)
 
print()
print('Recommended items:')
print(items_recommend_ids)
for item_id in items_recommend_ids:
    print (items[items.index == item_id].title)

Last best items are:
6876    Goya Guava Jelly, 17-Ounce Glass Jars (Pack of 3)
Name: title, dtype: object
36214    Calbee Potato Chips Seaweed/Salt, 2.8-Ounce Un...
Name: title, dtype: object
23664    Bemka.com American Sturgeon Hackleback Wild Ca...
Name: title, dtype: object
18978    Simply Organic Garlic Vinaigrette Dressing, 1-...
Name: title, dtype: object
36885    Excalibur Cinnamon, Ground, 4.25-Pound Shaker ...
Name: title, dtype: object

Recommended items:
{7555, 36228, 5768, 36750, 36899, 19876, 17573, 37418, 24235, 35498, 20908, 26300, 23612, 30782, 9547, 16332, 6095, 19543, 28506, 21212, 6882, 19558, 25325, 23671, 7422}
7555    McCormick Gourmet Whole Nutmeg, 1.5 oz
Name: title, dtype: object
36228    Yamamoto Green Tea (with roasted brown rice) 48g
Name: title, dtype: object
5768    Potato Flakes/Granules, 1 lb.
Name: title, dtype: object
36750    Lakewood Organic Fresh Pressed Pure Pineapple ...
Name: title, dtype: object
36899    Kidsmania Sour Flush Candy Plunger with S

In [46]:
# сохраним эмбеддинги для прототипа
import pickle
with open('items_embeddings.pkl', 'wb') as f_out:
    pickle.dump(item_embeddings, f_out)

# 5 Выводы

1 Построена базовая модель

2 Подобраны параметры модели с целью увеличить RocAuc, добавлена регуляризация и признаки товаров: скор ухудшился на моей тестовой выборке с 0.725 до 0.713, на Каггле стал хуже не значительно (с 0.76 до 0.75).

3 Добавлены признаки товаров. Но результат странный: добавление признаков в модель приводит к ухудшению RocAuc (с 0.76 до 0.74 ухудшается при добавлении фич)

4 Даны рекомендации пользователю на основе эмбеддингов понравившихся товаров

5. Для новых пользователей мойжно устраивать небольшй опрос при регистрации, а также пробовать рекомендовать самые популярные товары.
